In [ ]:
import polars as pl
lf = pl.scan_parquet('parquets/*.parquet')
print(f"Total sequences: {lf.select(pl.len().alias('count')).collect().item()}")

# VH - VL Pairing Preferences

In [ ]:
counts = lf.group_by(pl.col("v_call_light", "v_call_heavy")).agg(pl.len()).collect()

dfpandas = counts.to_pandas()

contab = dfpandas.pivot_table(index='v_call_heavy', columns='v_call_light', values='len').fillna(0)

In [ ]:
contab = contab.loc[contab.sum(1).gt(100),contab.sum(0).gt(100)]
total_seqs = contab.values.sum()

In [ ]:
hc_totals = contab.sum(axis=1).values

lc_totals = contab.sum(axis=0).values 

expected = np.outer(hc_totals, lc_totals) / total_seqs

In [ ]:
chisq_stat = ((contab - expected) ** 2 / expected).values.sum()

In [ ]:
chisq_stat

In [ ]:
from scipy.stats import chi2

In [ ]:
degfreedom = (contab.shape[0] - 1) * (contab.shape[1] - 1)
pval = 1 - chi2.cdf(chisq_stat, degfreedom)